## Question 1: **Answer**

In [1]:
import pandas as pd
import sqlite3

In [ ]:
def add_data_to_database(iphone_revenue, iphone_units):
    try:
        # Creating DataFrames from CSV files
        df_revenue = pd.read_csv(iphone_revenue)
        df_units = pd.read_csv(iphone_units)
        # Connecting to database
        engine = sqlite3.connect("apple_data.db")
        df_revenue.to_sql("iphone_revenue", con=engine, index = False)
        df_units.to_sql("iphone_units", con=engine, index = False)
        engine.close()
    except ValueError:
            print("Table already exists")

# Loading data
iphone_revenue = "/content/apple_iphone_revenue.csv"
iphone_units =  "/content/apple_iphone_unit_sales.csv"

add_data_to_database(iphone_revenue, iphone_units)

In [3]:
# Loading Magic extension
%load_ext sql

In [4]:
# Connecting to the database
%sql sqlite:///apple_data.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM iphone_revenue
LIMIT 5;

 * sqlite:///apple_data.db
Done.


ID,category,iPhone_revenue_billion,YOY growth
101,Q2/07,0.141,None
102,Q3/07,0.489,None
103,Q4/07,1.036,None
104,Q1/08,0.817,None
105,Q2/08,0.483,242.55


In [7]:
%%sql
SELECT * FROM iphone_units
LIMIT 5;

 * sqlite:///apple_data.db
Done.


category,iPhone_units_millions,YOY growth,sales_id
Q2/07,0.27,None,101
Q3/07,1.119,None,102
Q4/07,2.315,None,103
Q1/08,1.703,None,104
Q2/08,0.717,165.56,105


## Question 1: **Part B Answer**

In [8]:
%%sql
ALTER TABLE iphone_units
    RENAME 'YOY growth' TO 'yoy_growth';

 * sqlite:///apple_data.db
Done.


[]

In [9]:
%%sql
ALTER TABLE iphone_revenue
    RENAME 'YOY growth' TO 'yoy_growth';

 * sqlite:///apple_data.db
Done.


[]

## Question 2: **Answer**

In [10]:
%%sql
-- CTE to join and aggregate units and revenue
WITH min_units_and_revenue AS (
    SELECT
        ir.category,
        SUM(iu.iphone_units_millions) AS total_units,
        SUM(ir.iphone_revenue_billion) AS total_revenue
    FROM iphone_revenue ir
    INNER JOIN iphone_units iu
    -- joining tables on respective IDS
    ON ir.ID = iu.sales_id
    GROUP BY ir.category)

-- Main query returns the quarter with min units & revenue
SELECT
    category,
    total_units AS min_units,
    total_revenue AS min_revenue
FROM min_units_and_revenue
WHERE total_units = (SELECT MIN(total_units)
                    FROM min_units_and_revenue) AND
                    total_revenue = (SELECT MIN(total_revenue)
                                     FROM min_units_and_revenue)

 * sqlite:///apple_data.db
Done.


category,min_units,min_revenue
Q2/07,0.27,0.141


## Question 3: **Answer**

In [11]:
%%sql
-- filter rows for quarter 4 and
WITH quarterly_revenue AS (
    SELECT
        category,
        CASE
            WHEN category LIKE '%Q4%' THEN 'Q4'
            WHEN category LIKE '%Q1%' THEN 'Q1'
            ELSE NULL
        END AS quarter,
        SUM(iphone_revenue_billion) AS total_revenue
    FROM iphone_revenue
    WHERE category LIKE '%Q4%' OR category LIKE '%Q1%'
    -- Filter for the period 2007-2015 by extracting the year from the category
    AND CAST(SUBSTR(category, 4, 2) AS INTEGER) BETWEEN 7 AND 15
    GROUP BY category, quarter
),
-- Calculate total revenue for each quarter
quarter_comparison AS (
    SELECT
        ROUND(SUM(CASE WHEN quarter = 'Q4' THEN total_revenue ELSE 0 END), 2)
        AS quarter_4_sales,
        ROUND(SUM(CASE WHEN quarter = 'Q1' THEN total_revenue ELSE 0 END), 2)
        AS quarter_1_sales
    FROM quarterly_revenue
)
-- Returning Q4 and Q1 sales and the difference
SELECT
    quarter_4_sales,
    quarter_1_sales,
    ROUND((quarter_4_sales - quarter_1_sales), 2) AS difference
FROM quarter_comparison;

 * sqlite:///apple_data.db
Done.


quarter_4_sales,quarter_1_sales,difference
209.72,132.32,77.4


## Question 4: **Answer**

In [12]:
%%sql
SELECT
    ir.category,
    iu.iphone_units_millions,
    ir.iphone_revenue_billion,
    ir.yoy_growth AS revenue_growth
FROM iphone_revenue ir
INNER JOIN iphone_units iu
ON iu.sales_id = ir.ID
-- Exclude rows with NULL values in revenue_growth
WHERE revenue_growth IS NOT NULL
LIMIT 10;

 * sqlite:///apple_data.db
Done.


category,iPhone_units_millions,iPhone_revenue_billion,revenue_growth
Q2/08,0.717,0.483,242.55
Q3/08,6.892,4.406,801.02
Q4/08,4.363,2.94,183.78
Q1/09,3.793,2.427,197.06
Q2/09,5.208,3.06,533.54
Q3/09,7.367,4.606,4.54
Q4/09,8.737,5.578,89.73
Q1/10,8.752,5.445,124.35
Q2/10,8.398,5.334,74.31
Q3/10,14.102,8.822,91.53


## Question 5: **Answer**

In [13]:
%%sql
-- Calculate total revenue and units per quarter
WITH per_quarter_data AS (
    SELECT
        iphone_revenue.category AS quarter,
        iphone_revenue.yoy_growth,
        SUM(iphone_revenue.iphone_revenue_billion)
        AS total_revenue,
        SUM(iphone_units.iphone_units_millions)
        AS total_units
    FROM iphone_revenue
    INNER JOIN iphone_units
    ON iphone_revenue.ID = iphone_units.sales_id
    WHERE iphone_revenue.yoy_growth IS NOT NULL
    GROUP BY iphone_revenue.category)
-- Returning quarter with minimum growth
SELECT
    pqd.quarter,
    MIN(pqd.yoy_growth) AS yoy_growth,
    pqd.total_revenue,
    pqd.total_units
FROM per_quarter_data pqd
INNER JOIN per_quarter_data pqd2
ON pqd.yoy_growth = pqd2.yoy_growth;

 * sqlite:///apple_data.db
Done.


quarter,yoy_growth,total_revenue,total_units
Q2/16,-23.34,24.048,40.399


In [14]:
# %%sql
# DROP TABLE iphone_revenue;

In [15]:
# %%sql
# DROP TABLE iphone_units;